In [2]:
!pip3 install pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import pickle
import IPython
import IPython.display
from keras.models import load_model
import matplotlib.pyplot as plt
from functions.naming import rename_columns

# Load the data
df = pd.read_excel("/Users/athanasioskaravangelis/Desktop/RSM BAM/Workshop/pko_forecasting/data/PKO_Initial_Dataset.xlsx")
#rename columns
df = rename_columns(df)
# select only the values after 2010-01-01
df['date'] = pd.to_datetime(df['date'], format='%b-%y')
df = df[df['date'] > '2009-12-02']
#set date as index
df = df.set_index('date')

In [51]:
# Constants
MAX_EPOCHS = 20
INPUT_WINDOW = 12
#NUM_FEATURES = 9  # This will be updated based on the filtered dataset

In [66]:
def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=patience,mode='min')
  model.compile(loss=tf.losses.MeanSquaredError(),optimizer=tf.keras.optimizers.legacy.Adam(),metrics=[tf.metrics.MeanSquaredError()])
  history = model.fit(window.train, epochs=MAX_EPOCHS, validation_data=window.val, callbacks=[early_stopping], verbose=0)
  return history

In [67]:
# Specify columns to exclude
exclude_columns = [
    'pko_total_supply_malaysia', 'indonesia_disaster', 'malaysia_disaster',
    'pko_fob_malaysia', 'jet_fuel_us', 'jet_fuel_europe', 'soybean_oil_zlz2',
    'tallow_fob_us_gulf', 'bio_ethanol', 'rspo', 'palm_oil_cif_nwe', 'palm_olein_fob_malaysia',
    'palm_stearin_cif_rotterdam', 'fatty_alcohol_c12_14_fob_asia', 'fatty_alcohol_c16_18_fob_asia',
    'fatty_alcohol_c12_14_fd_nwe', 'jet_fuel_us_usd_mt'
]

# Filter the dataset to exclude specified columns
df_filtered = df.drop(columns=exclude_columns)

# Update NUM_FEATURES based on the filtered dataset
num_features = df_filtered.shape[1]
num_features

# fill nas with average values
df_filtered = df_filtered.fillna(df_filtered.mean())

In [68]:
df_filtered.shape

(167, 35)

In [80]:
from sklearn.discriminant_analysis import StandardScaler


def train():
    X = df_filtered

    # Check and handle infinity and large values
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    X.fillna(X.replace([np.inf, -np.inf]), inplace=True)

    # Print information about NaN and infinite values
    print("NaN values in X:", X.isna().sum().sum())
    print("Infinite values in X:", np.isinf(X).sum().sum())

    X.replace([np.inf, -np.inf], np.finfo(np.float64).max, inplace=True)
    # Split the data into training, validation, and testing sets
    train_df = X[0:133]
    val_df = X[133:156]
    test_df = X[156:166]

    n = len(df)

    # Normalize the data
    train_mean = train_df.mean()
    train_std = train_df.std()
    train_df = (train_df - train_mean) / train_std
    val_df = (val_df - train_mean) / train_std
    test_df = (test_df - train_mean) / train_std

    class WindowGenerator():
            def __init__(self, input_width, label_width, shift,train_df=train_df, val_df=val_df, test_df=test_df,label_columns=None):

                self.train_df = train_df
                self.val_df = val_df
                self.test_df = test_df
                self.label_columns = label_columns
                if label_columns is not None:
                    self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
                self.column_indices = {name: i for i, name in enumerate(train_df.columns)}
                self.input_width = input_width
                self.label_width = label_width
                self.shift = shift
                self.total_window_size = input_width + shift
                self.input_slice = slice(0, input_width)
                self.input_indices = np.arange(self.total_window_size)[self.input_slice]
                self.label_start = self.total_window_size - self.label_width
                self.labels_slice = slice(self.label_start, None)
                self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

                assert label_width <= shift
                assert shift > 0

            def split_window(self, features, target):
                if len(features.shape) == 2:
                    features = features[:, :, np.newaxis]

                if len(target.shape) == 0:
                    target = target[np.newaxis, np.newaxis, np.newaxis]
                elif len(target.shape) == 1:
                    target = target[:, np.newaxis, np.newaxis]

                inputs = features[:, :-1, :]
                labels = target[:, :, np.newaxis]
                return inputs, labels


            # old version
            """def split_window(self, features):
                inputs = features[:, self.input_slice, :]
                labels = features[:, self.labels_slice, :]
                if self.label_columns is not None:
                    labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns],axis=-1)

                inputs.set_shape([None, self.input_width, None])
                labels.set_shape([None, self.label_width, None])
                return inputs, labels"""

            def plot(self,model=None,train_mean=0,train_std=0,plot_col='pko_fob_malaysia', max_subplots=5):
                inputs, labels = self.example
                plt.figure(figsize=(12, 8))
                plot_col_index = self.column_indices[plot_col]
                max_n = min(max_subplots, len(inputs))
                for n in range(max_n):
                    plt.subplot(max_n, 1, n+1)
                    inputs_n = inputs[n, :, plot_col_index] * train_std[plot_col_index] + train_mean[plot_col_index]
                    plt.plot(self.input_indices,inputs_n,label='Inputs', marker='.', zorder=-10)

                    if self.label_columns:
                        label_col_index = self.label_columns_indices.get(plot_col, None)
                    else:
                        label_col_index = plot_col_index
                    if label_col_index is None:
                        continue

                    labels_n = labels[n, :, label_col_index] * train_std[plot_col_index] + train_mean[plot_col_index]
                    plt.scatter(self.label_indices, labels_n,edgecolors='k', label='Labels', c='#2ca02c', s=64)
                    if model is not None:
                        predictions = model(inputs)
                        predictions = predictions[n, :, label_col_index] * train_std[plot_col_index] + train_mean[plot_col_index]
                        print('line 102')
                        print (label_col_index,plot_col_index)
                        plt.scatter(self.label_indices,predictions,marker='X', edgecolors='k', label='Predictions',c='#ff7f0e', s=64)

                    if n == 0:
                        plt.legend()
            
                plt.xlabel('Timeslots [month]')

            def make_dataset(self, data):
                target_variable = 'pko_cif_rotterdam'

                # Extract the target variable and features
                target = data[target_variable].values
                features = data.drop(columns=[target_variable]).values

                # Standardize the features using StandardScaler
                scaler = StandardScaler()
                features_scaled = scaler.fit_transform(features)

                # Create sequences for input data and target data
                X_sequences = []
                y_targets = []

                for i in range(len(features_scaled) - self.total_window_size + 1):
                    X_sequences.append(features_scaled[i:i+self.total_window_size, :])
                    y_targets.append(target[i+self.total_window_size-1])

                # Convert to numpy arrays
                X_sequences = np.array(X_sequences)
                y_targets = np.array(y_targets)

                # Create a TensorFlow dataset directly from NumPy arrays
                ds = tf.data.Dataset.from_tensor_slices((X_sequences, y_targets))
                ds = ds.map(self.split_window)
                
                return ds


            
            @property
            def train(self):
                return self.make_dataset(self.train_df)

            @property
            def val(self):
                return self.make_dataset(self.val_df)

            @property
            def test(self):
                return self.make_dataset(self.test_df)

            @property
            def example(self):
                result = getattr(self, '_example', None)
                if result is None:
                    result = next(iter(self.train))
                    self._example = result
                return result
    OUT_STEPS = 12
    multi_window = WindowGenerator(input_width=12,label_width=OUT_STEPS,shift=OUT_STEPS, train_df=train_df,val_df=val_df,test_df=test_df)
    multi_val_performance = {}
    multi_performance = {}

    multi_lstm_model = tf.keras.Sequential([
            tf.keras.layers.LSTM(32, return_sequences=False),
            tf.keras.layers.Dense(OUT_STEPS* num_features,kernel_initializer=tf.initializers.zeros()),
            tf.keras.layers.Reshape([OUT_STEPS, num_features])
        ])

    history = compile_and_fit(multi_lstm_model, multi_window)

    IPython.display.clear_output()

    multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
    multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
    multi_window.plot(multi_lstm_model,train_mean,train_std)
        
    plt.show()
    return multi_lstm_model,train_mean,train_std,n

multi_lstm_model,train_mean,train_std,n= train()

/var/folders/35/tbpcvh252s9dy7dhvwv499800000gn/T/ipykernel_2205/1146248076.py:9: FutureWarning: DataFrame.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  X.fillna(X.replace([np.inf, -np.inf]), inplace=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


NaN values in X: 0
Infinite values in X: 0


ValueError: Input X contains infinity or a value too large for dtype('float64').

In [18]:
def predict(multi_lstm_model,n):
    label_col_index = 7
    plot_col_index = 7
    predictions = multi_lstm_model
    predictions = predictions[n, :, label_col_index] * train_std[plot_col_index] + train_mean[plot_col_index]
    return predictions

In [19]:
k = predict(multi_lstm_model,n)

NameError: name 'multi_lstm_model' is not defined